## Train the Model With One Layer on Top of CamemBERT

In [ ]:
import os
import numpy as np

from silence_tensorflow import silence_tensorflow
silence_tensorflow()  # silence TF warnings
import tensorflow as tf

from utils import loadFile
from dataProcessing import encodeData, insertTarget, processingScriber, processingOPUS

from transformers import AutoTokenizer, TFCamembertForMaskedLM
from datetime import datetime
import json
import sys

In [ ]:
### instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained("jplu/tf-camembert-base", do_lower_case=True)

### punctuation encoder
punctuationEnc = {
    'SPACE': 0,
    'PERIOD': 1,
}

### Set Vocabulary Size
vocabSize = 32005

### hyper-parameters
sequenceSize = 32
batchSize = 32
learningRate = 5e-6
trainLayerIndex = 0
numEpo = 3

listHyper0 = ['vocabSize', 'sequenceSize', 'batchSize', 'learningRate', 'trainLayerIndex', 'numEpo']
listHyper1 = [str(vocabSize), str(sequenceSize), str(batchSize), str(learningRate), str(trainLayerIndex), str(numEpo)]
time = datetime.now().strftime("%Y%m%d_%H%M%S")
save_path = 'ModelsExpScriber/{}/'.format(time)
os.mkdir(save_path)

## Build The Dataset

In [ ]:
### Training Dataset
print('\nProcessing Data ... ')

# THIS IN CASE STARTING FROM FILE WITH SENTENCES
# this is the file structured in sentences
trainDataName = "./DataScriber/raw.processed.Train_01.txt"
# from sentences to columns words+punctuation
dataTrain = loadFile(processingScriber(trainDataName))

# # THIS IN CASE STARTING FROM FILE WITH COLUMNS
# trainDataName = './AudioFeatures/outFile_05.txt'
# dataTrain = loadFile(trainDataName)

### Encode Data and insert target
XTrain, yTrain = encodeData(dataTrain, tokenizer, punctuationEnc)

### Create Sequences With The Target
XTrainMod = insertTarget(XTrain, sequenceSize)

# build the datasets
# trainDataset = tf.data.Dataset.from_tensor_slices((XTrainMod, yTrain)).batch(batchSize)
trainDataset = tf.data.Dataset.from_tensor_slices((XTrainMod, yTrain)).shuffle(buffer_size=1000000).batch(batchSize)

print("\nTraining Dataset Tensor Shape = ", XTrainMod.shape)

### Build The Model

In [ ]:
print('\nBulding the Model ... ')

bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bert_input')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(bert_input)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
dense_out = tf.keras.layers.Dense(len(punctuationEnc))(x)

model = tf.keras.Model(bert_input, dense_out)

# define the loss
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
def loss(model, x, y):
    y_ = model(x)
    return loss_object(y_true=y, y_pred=y_)

# func to calculate the gradients
def grad(model, inputs, targets, trainLayerIndex):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables[trainLayerIndex:])

# define the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate)

### Load Weights From A Fine-Tuned Model

In [ ]:
# # checkpointPath = "./ModelsExpScriber/20200628_151812/cp-001.ckpt"
# checkpointPath = "./ModelsExpOPUS/20200729_101800/cp-003.ckpt"
# model.load_weights(checkpointPath)

### Study The Weights

In [ ]:
# print(len(model.layers))
# print(len(model.variables))

In [ ]:
# # Print word_embeddings Weights
# print(model.trainable_variables[194][0:])

In [ ]:
# ### Load Weigths From a Previous Experiment

# checkpointPath = "./Models/20200530_161559/cp-001.ckpt"  # this is the baseline model
# model.load_weights(checkpointPath)

In [ ]:
# # Print word_embeddings Weights
# print(model.trainable_variables[194][0:])

### Training Loop

In [ ]:
print("\nExperiment Folder: ", time)
print("\nHyperparameters:")
print('sequenceSize = ', sequenceSize)
print('batchSize = ', batchSize)
print('learningRate = ', learningRate)
print('train Layer Index = ', trainLayerIndex)
print('numEpo = ', numEpo)

epoch_loss_avg = tf.keras.metrics.Mean()
epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

train_loss_results = []
train_accuracy_results = []

checkpoint_path = save_path + "cp-{epoch:03d}.ckpt"

print("\nTraining the Model ... ")

tmpTrain = np.inf
for epoch in range(1, numEpo+1):

    # training loop
    for x, y in trainDataset:
        # optimize the model
        loss_value, grads = grad(model, x, y, trainLayerIndex)
        optimizer.apply_gradients(zip(grads, model.trainable_variables[trainLayerIndex:]))

        # track progress
        epoch_loss_avg.update_state(loss_value)
        epoch_accuracy.update_state(y, model(x))

    # end epoch
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())

    print("\nEpoch {:03d}: (Training)   Loss: {:.3f}, Accuracy: {:.3%}".format(epoch, epoch_loss_avg.result(), epoch_accuracy.result()))

    # # save model if new min for train loss is found
    # if epoch_loss_avg.result().numpy() < tmpTrain:
    tmpTrain = epoch_loss_avg.result().numpy()
    model.save_weights(checkpoint_path.format(epoch=epoch))

    epoch_loss_avg.reset_states()
    epoch_accuracy.reset_states()

## Output Training Details On Log File

In [ ]:
nameLogFile = 'log.txt'
logFile = open(save_path + nameLogFile, "w")

# write name of model
logFile.write("\n" + time + "\n\n")

# write hyper parameters
for i in range(len(listHyper0)):
    logFile.write(listHyper0[i] + ":  " + listHyper1[i] + "\n")

# write training details
logFile.write('\nTRAINING')
trainLossArr = np.asarray(train_loss_results)
trainAccArr = np.asarray(train_accuracy_results)
for i in range(numEpo):
    epoch = i+1
    logFile.write("\nEpoch {:03d}:   Loss: {:7.4f},   Accuracy: {:7.4%}".format(epoch, trainLossArr[i], trainAccArr[i]))

## Evaluate the Model, Write the Details on the logFile

In [ ]:
### Get the Test Dataset

# name of dataset with sentences
dataName = "./DataScriber/raw.processed.Test_01.txt"

# from sentences to columns words+punctuation
data = loadFile(processingScriber(dataName))

### Encode Data
X, y = encodeData(data, tokenizer, punctuationEnc)

### Create Sequences With The Target
XMod = insertTarget(X, sequenceSize)

# one hot encode the labels
yMod = tf.one_hot(y, len(punctuationEnc), dtype='int64').numpy()

dataBuilt = tf.data.Dataset.from_tensor_slices((XMod, yMod)).batch(batchSize)

print("\nTest Dataset Tensor Shape = ", XMod.shape)

In [ ]:
### build and compile model

bert_input = tf.keras.Input(shape=(sequenceSize), dtype='int32', name='bert_input')
x = TFCamembertForMaskedLM.from_pretrained("jplu/tf-camembert-base")(bert_input)[0]
x = tf.keras.layers.Reshape((sequenceSize*vocabSize,))(x)
dense_out = tf.keras.layers.Dense(len(punctuationEnc), activation='softmax')(x)

model = tf.keras.Model(bert_input, dense_out, name='model')

model.compile(optimizer='adam',
              loss=tf.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[tf.keras.metrics.Recall(class_id=0, name='Rec_0'),
                       tf.keras.metrics.Precision(class_id=0, name='Prec_0'),
                       tf.keras.metrics.Recall(class_id=1, name='Rec_1'),
                       tf.keras.metrics.Precision(class_id=1, name='Prec_1'),
                      ])

In [ ]:
### Get List of the Models in the Output Folder

modelsLst = []
for r, d, f in os.walk(save_path):
    for file in sorted(f):
        if ".index" in file:
            modelsLst.append(file[:-6])

In [ ]:
### Compute F1 Score

def compF1(rec, pre):
    if pre + rec == .0:
        return .0
    else:
        return 2 * (pre*rec) / (pre+rec)

In [ ]:
### Evaluate the Models

print("\nEvaluate Models")

logFile.write('\n\nEVALUATION\n')
for i in range(len(modelsLst)):
    checkpointPath = save_path + modelsLst[i]
    print(checkpointPath)

    # load weights
    model.load_weights(checkpointPath)

    # evaluate
    evaluation = model.evaluate(dataBuilt)
    
    f1_0 = compF1(evaluation[1],evaluation[2])
    f1_1 = compF1(evaluation[3],evaluation[4])
    print("F1_0 = {:10.7f} - F1_1 = {:10.7f}".format(f1_0, f1_1))
    
    # write details on log files
    logFile.write(modelsLst[i])
    logFile.write(" - Loss = {:7.4f} - Rec_0 = {:6.4f} - Pre_0 = {:6.4f} - F1_0 = {:10.7f} - Rec_1 = {:6.4f} - Pre_1 = {:6.4f} - F1_1 = {:10.7f}\n".format(evaluation[0], evaluation[1], evaluation[2], f1_0, evaluation[3], evaluation[4], f1_1))

logFile.close()